In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape, Conv2DTranspose, Activation, ReLU
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pdb
import os
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# path = r'D:\MILCOM_2025\Spec_BIN_data_SNR-5'
# path = r'D:\MILCOM_2025\Spec_BIN_data_SNR_0'
# path = r'D:\MILCOM_2025\Spec_BIN_data_SNR5'

In [ ]:
save_dir = r"D:\MILCOM_2025\Results"

In [ ]:
img_height = 256
img_width = 256
batch_size = 32

In [ ]:
# Step 1: Get all file paths and labels
all_files = sorted(glob(os.path.join(path, '*', '*')))
all_labels = [os.path.basename(os.path.dirname(f)) for f in all_files]
class_names = sorted(set(all_labels))
label_to_index = {label: idx for idx, label in enumerate(class_names)}
all_indices = [label_to_index[label] for label in all_labels]

# Step 2: Convert to numpy arrays
all_files = np.array(all_files)
all_indices = np.array(all_indices)

# Step 3: Strict split — 50% train, 15% val, 35% test (or any ratio you want)
X_temp, X_test, y_temp, y_test = train_test_split(all_files, all_indices, test_size=0.15, random_state=42, stratify=all_indices)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)  # 0.23 of 65% ≈ 15%

# Step 4: Helper to load and preprocess image
def preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image.set_shape((img_height, img_width, 3))
    label = tf.one_hot(label, depth=len(class_names))
    return image, label

# Step 5: Build tf.data.Dataset for each split
def build_dataset(file_paths, labels, is_train=True):
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if is_train:
        ds = ds.shuffle(1000)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = build_dataset(X_train, y_train, is_train=True)
val_ds = build_dataset(X_val, y_val, is_train=False)
test_ds = build_dataset(X_test, y_test, is_train=False)


In [ ]:
# Confirm no overlaps across splits
def assert_disjoint(a, b, name1, name2):
    overlap = set(a).intersection(set(b))
    assert len(overlap) == 0, f"Data leakage detected between {name1} and {name2}"

assert_disjoint(X_train, X_val, "train", "val")
assert_disjoint(X_train, X_test, "train", "test")
assert_disjoint(X_val, X_test, "val", "test")
print("✅ All splits are 100% disjoint — no data leakage.")


In [ ]:
class_names = sorted(set(all_labels))
print(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
num_classes = len(class_names)

## Without Normalization

In [ ]:
model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(256, 256, 3)),
  layers.MaxPooling2D(),

  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Conv2D(128, 3, padding='same', activation='relu'), 
  layers.MaxPooling2D(),

  layers.Flatten(),

  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')  # Adjust class count if needed
])


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100)

# Inference Time Calculation

In [ ]:
# import time

# # Take one image from test_ds
# for images, labels in test_ds.take(1):
#     sample_image = images[0:1]
#     break

# # Warm up
# _ = model.predict(sample_image)

# # Run inference 1000 times and measure time
# total_time = 0
# num_runs = 1000
# for _ in range(num_runs):
#     start = time.time()
#     _ = model.predict(sample_image, verbose=0)
#     end = time.time()
#     total_time += (end - start)

# # Compute average inference time in milliseconds
# avg_inference_time_ms = (total_time / num_runs) * 1000
# print(f"✅ Average inference time over {num_runs} runs: {avg_inference_time_ms:.3f} ms")


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import tensorflow as tf

# Step 1: Create feature extractor from trained model
feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

# Step 2: Extract features and labels from test_ds
features = []
labels = []

for images, label_batch in test_ds:
    feats = feature_extractor(images)
    features.append(feats.numpy())
    labels.append(tf.argmax(label_batch, axis=1).numpy())

features = np.concatenate(features, axis=0)
labels = np.concatenate(labels, axis=0)

# Step 3: Apply 3D t-SNE
tsne_3d = TSNE(n_components=3, perplexity=30, n_iter=1000, random_state=42).fit_transform(features)

# Step 4: Define different angles (elevation, azimuth)
view_angles = [
    (20, 30),    # Default front view
    (30, 120),   # Left rotated and above
    (10, 210),   # Side view
    (25, 300)    # Rear rotated
]

# Step 5: Create multi-angle subplots
fig = plt.figure(figsize=(20, 16))
palette = sns.color_palette("hsv", len(class_names))

for i, (elev, azim) in enumerate(view_angles):
    ax = fig.add_subplot(2, 2, i + 1, projection='3d')
    for j, class_name in enumerate(class_names):
        idx = labels == j
        ax.scatter(tsne_3d[idx, 0], tsne_3d[idx, 1], tsne_3d[idx, 2],
                   label=class_name if i == 0 else "", s=40, edgecolor='black')
    ax.view_init(elev=elev, azim=azim)
    ax.set_title(f"View {i+1} (elev={elev}, azim={azim})", fontsize=12, fontweight='bold')
    ax.set_xlabel("TSNE-1", fontsize=10, fontweight='bold')
    ax.set_ylabel("TSNE-2", fontsize=10, fontweight='bold')
    ax.set_zlabel("TSNE-3", fontsize=10, fontweight='bold')
    ax.tick_params(axis='both', labelsize=8)

# Add legend only once
handles, legend_labels = ax.get_legend_handles_labels()
fig.legend(handles, legend_labels, loc='upper center', ncol=len(class_names), fontsize=10)

plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the plot (optional)
# plt.savefig(r"D:\MILCOM_2025\Results\tsne_3D_views_multiangle.png", dpi=600)

plt.show()


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import tensorflow as tf

# Step 1: Create feature extractor from trained model
feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

# Step 2: Extract features and labels from test_ds
features = []
labels = []

for images, label_batch in test_ds:
    feats = feature_extractor(images)
    features.append(feats.numpy())
    labels.append(tf.argmax(label_batch, axis=1).numpy())

features = np.concatenate(features, axis=0)
labels = np.concatenate(labels, axis=0)

# Step 3: Apply 3D t-SNE
tsne_3d = TSNE(n_components=3, perplexity=30, max_iter=1000, random_state=42).fit_transform(features)



In [ ]:
# Step 4: Plot only View 2 (elev=30, azim=120)
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')
palette = sns.color_palette("hsv", len(np.unique(labels)))

for i, class_name in enumerate(class_names):
    idx = labels == i
    ax.scatter(tsne_3d[idx, 0], tsne_3d[idx, 1], tsne_3d[idx, 2],
               label=class_name, s=20)

ax.view_init(elev=18, azim=107)
# ax.set_title("t-SNE Projection", fontsize=12, fontweight='bold')
ax.set_xlabel("Component 1", fontsize=10, fontweight='bold')
ax.set_ylabel("Component 2", fontsize=10, fontweight='bold')
ax.set_zlabel("Component 3", fontsize=10, fontweight='bold')

ax.tick_params(axis='both', labelsize=10)
ax.tick_params(axis='z', labelsize=10) 
# after your scatter() calls, but before plt.show()
handles, _ = ax.get_legend_handles_labels()

my_labels = ['5G', 'Radar', 'Radar and 5G', ...] 
fig.legend(handles, my_labels,
           loc='upper left',
           bbox_to_anchor=(0.15, 0.88),
           ncol=1,
           fontsize=10)
plt.tight_layout()

plt.savefig(r"D:\MILCOM_2025\Results\tsne for CNN Radar Detection data_New_binary.pdf",  dpi=600, bbox_inches='tight')
plt.show()


plt.show()

In [ ]:
# model.save(r'D:\SDRChallenge-main\Demo\DemoCNN.h5')
# model = load_model(r'D:\SDRChallenge-main\Demo\DemoCNN.h5')

In [ ]:
# save_dir = r"D:\SDRChallenge-main\Plots\Training_from_scratch_on_half_real_data"  # Your desired folder path

In [ ]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

In [ ]:
# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 6))

# Plot the accuracy curves
plt.plot(training_accuracy, label='Training Accuracy', linestyle='--', linewidth=2)  # Thicker dotted curve
plt.plot(validation_accuracy, label='Validation Accuracy', linewidth=2)  # Thicker solid curve

# Adding labels, title, and legend with larger and bold font
plt.xlabel('Epochs', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy', fontsize=14, fontweight='bold')
plt.title('Training and Validation Accuracy', fontsize=16, fontweight='bold')

# Make the legend font size larger and bold using 'prop'
plt.legend(fontsize=12, prop={'weight':'bold'})

# Make tick labels bold and increase font size
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Adding a grid with both major and minor grid lines
plt.grid(True, which='both', linewidth=0.3)  # Smaller squares with thinner grid lines
plt.minorticks_on()  # Enable minor ticks
plt.tight_layout()
# plt.savefig(os.path.join(save_dir, "training_val_accuracy_curve_SNR-5_data_CNN.png"), dpi=600)

# Display the plot
plt.show()

In [ ]:
# Assuming that 'history' is the result of model training
training_loss = history.history['loss']
validation_loss = history.history['val_loss']


# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 6))

# Plot the accuracy curves
plt.plot(training_loss, label='Training Loss', linestyle='--', linewidth=2)  # Thicker dotted curve
plt.plot(validation_loss, label='Validation Loss', linewidth=2)  # Thicker solid curve

# Adding labels, title, and legend with larger and bold font
plt.xlabel('Epochs', fontsize=14, fontweight='bold')
plt.ylabel('Loss', fontsize=14, fontweight='bold')
plt.title('Training and Validation Loss', fontsize=16, fontweight='bold')

# Make the legend font size larger and bold using 'prop'
plt.legend(fontsize=12, prop={'weight':'bold'})

# Make tick labels bold and increase font size
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Adding a grid with both major and minor grid lines
plt.grid(True, which='both', linewidth=0.3)  # Smaller squares with thinner grid lines
plt.minorticks_on()  # Enable minor ticks
plt.tight_layout()
# plt.savefig(os.path.join(save_dir, "training_val_loss_SNR-5_data_CNN.png"), dpi=600)
# Display the plot
plt.show()

In [ ]:
# Step 1: Make predictions on the validation dataset
# Get the true labels and the predicted labels
y_true = []
y_pred = []

for images, labels in test_ds:
    y_true.extend(np.argmax(labels.numpy(), axis=1))  # True labels
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))  # Predicted labels

# Convert to numpy arrays for confusion matrix
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# radar_idx = class_names.index('Radar')
# radar_5g_idx = class_names.index('5G_Radar')
# Only5g_idx = class_names.index('5G')
# Noise_idx = class_names.index('Noise')


# # Step 4: Count the number of samples predicted as 'radar' and 'radar + 5G'
# radar_count = (y_pred == radar_idx).sum()
# radar_5g_count = (y_pred == radar_5g_idx).sum()

# Calculate the total radar-related samples (radar + radar and 5G)
# total_radar_count = radar_count + radar_5g_count

# Print the results
# print(f"{radar_count} samples have radar signal.")
# print(f"{radar_5g_count} samples have radar and 5G signal.")
# print(f"{total_radar_count} samples have radar-related signal (including radar and radar + 5G).")



In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 7))

# Increase annotation size and make them bold
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, cbar=False, 
            annot_kws={"size": 20, "fontweight": "bold"})  # Set annotation font size and bold


# Increase font size for axis labels and title
plt.xlabel('Predicted Label ', fontsize=16, fontweight = 'bold')
plt.ylabel('True Label', fontsize=16, fontweight = 'bold')

# Additional step: Use plt.setp() to make tick labels bold
plt.setp(plt.gca().get_xticklabels(), fontweight='bold')  # Set x-tick labels to bold
plt.setp(plt.gca().get_yticklabels(), fontweight='bold')  # Set y-tick labels to bold

# Increase tick label font size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Save the figure with 400 DPI
plt.tight_layout()
# plt.savefig(os.path.join(save_dir, "confusion_matrix_SNR-5_data_CNN.png"), dpi=600)

plt.show()

In [ ]:
# Compute and normalize confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Create custom annotation: percentage with % for non-zeros, '0' for zeros
annot = np.empty_like(cm_percent, dtype=object)
for i in range(cm_percent.shape[0]):
    for j in range(cm_percent.shape[1]):
        if cm_percent[i, j] == 0:
            annot[i, j] = '0'
        else:
            annot[i, j] = f"{cm_percent[i, j]:.1f}%"

# Plot heatmap
plt.figure(figsize=(8, 7))
sns.heatmap(cm_percent, annot=annot, fmt='', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, cbar=False,
            annot_kws={"size": 20, "fontweight": "bold"})

plt.xlabel('Predicted Label', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=16, fontweight='bold')
plt.setp(plt.gca().get_xticklabels(), fontweight='bold', fontsize=14)
plt.setp(plt.gca().get_yticklabels(), fontweight='bold', fontsize=14)
plt.tight_layout()
# plt.savefig(os.path.join(save_dir, "confusion_matrix_SNR-5_data_CNN_in_percentage.png"), dpi=600)
plt.show()


In [ ]:
misclassified_indices = np.where(y_true != y_pred)[0]
print("Misclassified sample indices:", misclassified_indices)


## Seperating the Test

In [ ]:
# test_path = r'D:\SDRChallenge-main\SDR_data\Binary_SNR-10\Binary SNR-10 hough transform-test'